In [64]:
import pandas as pd
import sklearn as skl

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from definitions import *
from dataset_helper_functions import *

Global switches.

In [65]:
try:
    IS_MASTER
except: 
    IS_MASTER = False

Load data.

In [71]:
if not IS_MASTER:
    data, embeddings, features = {}, {}, {}
    emb_model = 'all-mpnet-base-v2'#'multi-qa-mpnet-base-dot-v1'#'all-MiniLM-L6-v2'
    
    dev_path = p.join(PROC_DATA_DIR_PATH, 'dev')
    features_path = p.join(PROC_DATA_DIR_PATH, 'features')
    embeddings_path = p.join(PROC_DATA_DIR_PATH, 'embeddings')

    data_paths = {
        'dev': [
            p.join(dev_path, 'dev.tsv'),
            p.join(dev_path, 'dev_spacy.pkl'),
            p.join(embeddings_path, f'dev_sent_emb_{emb_model}.pkl'),
            p.join(features_path, 'dev_stylometric_features.pkl'),
        ],
        'test': [
            p.join(POLIT_DATA_DIR_PATH, 'test', 'test_combined.tsv'),
            p.join(PROC_DATA_DIR_PATH, 'test', 'test_spacy.pkl'),
            p.join(embeddings_path, f'test_sent_emb_{emb_model}.pkl'),
            p.join(features_path, 'test_stylometric_features.pkl'),

        ],
        'train': [
            p.join(POLIT_DATA_DIR_PATH, 'train', 'train_combined.tsv'),
            p.join(PROC_DATA_DIR_PATH, 'train', 'train_spacy.pkl'),
            p.join(embeddings_path, f'train_sent_emb_{emb_model}.pkl'),
            p.join(features_path, 'train_stylometric_features.pkl')
        ],
        # 'val': [
        #     p.join(POLIT_DATA_DIR_PATH, 'val', 'val_combined.tsv'),
        #     p.join(PROC_DATA_DIR_PATH, 'val', 'val_spacy.pkl'),
        #     p.join(embeddings_path, f'val_sent_emb_{emb_model}.pkl'),
        #     p.join(features_path, 'val_stylometric_features.pkl'),
        # ],
    }

    for dtype, dpaths in data_paths.items():
        try:
            data[dtype] = pd.read_pickle(dpaths[1])
            embeddings[dtype] = pd.read_pickle(dpaths[2])
            features[dtype] = pd.read_pickle(dpaths[3])
        except Exception as e:
            print(e.args)
            exit()

Concat embeddings with features.

In [72]:
feature_specifier = 'wstop_count'
# print(data)
# df = data['train']
# df['label'].describe()
# df.loc[np.array(list(map(len, df['label'].values)))>1]

In [73]:
input_vectors = {}
scaler = MinMaxScaler()
# concat 
for dtype in data_paths:
    emb = embeddings[dtype].iloc[:, 1:].values
    # print(len(emb))
    scaler.fit(emb)
    emb_scaled = scaler.transform(emb)

    feat = features[dtype]
    feat_cols = []
    for col in feat.columns:
        if feature_specifier in col:
            feat_cols.append(col)

    # print(feat[feat_cols])
    input_vectors[dtype] = np.concatenate((emb_scaled, feat[feat_cols].values), axis=1)

Set-up model

In [79]:
from sklearn.metrics import precision_score

first_layer_size = input_vectors['train'].shape[1]
print(first_layer_size)
# first_layer_size, 600, 200, 20, 1
model = MLPClassifier(
    hidden_layer_sizes=(400,),
    # validation_fraction=0.2,
    max_iter=600,
    # early_stopping=True
)

model.fit(input_vectors['train'], data['train']['label'].values)
pred = model.predict(input_vectors['test']).astype(np.int64)

# print(model.loss_)

858


In [78]:
# pred = [int(x) for x in pred]
# pred = pred.astype(np.int64)
print(pred)
print(data['test']['label'].values)
print('precision: ', precision_score(data['test']['label'].values, pred))
print('acc: ', model.score(input_vectors['test'], data['test']['label'].values))

[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
precision:  0.0
acc:  0.9790058660080272


/Users/icobx/Documents/skola/dp/venv_dp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
